In [245]:
import requests, pandas as pd, json, os, importlib, altair as alt

import sys
sys.path.append('../')

import altair_wrapper.eco_styles as eco_styles
styles = eco_styles.EcoStyles()

House Price Index Datasets: [Link](https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index-Datasets.aspx#mpo)

In [5]:
hpi_master = 'https://www.fhfa.gov/HPI_master.csv'

df_raw = pd.read_csv(hpi_master)

df_raw

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,1,100.00,100.00
1,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,2,100.91,100.97
2,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,3,101.31,100.93
3,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,4,101.70,100.99
4,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,5,102.33,101.37
...,...,...,...,...,...,...,...,...,...,...
123965,developmental,purchase-only,quarterly,Puerto Rico,Puerto Rico,PR,2022,3,195.75,194.01
123966,developmental,purchase-only,quarterly,Puerto Rico,Puerto Rico,PR,2022,4,192.22,194.88
123967,developmental,purchase-only,quarterly,Puerto Rico,Puerto Rico,PR,2023,1,189.95,193.02
123968,developmental,purchase-only,quarterly,Puerto Rico,Puerto Rico,PR,2023,2,212.54,209.17


Let's download the complementary files for the main HPI dataset.

In [23]:
dictionary_url = 'https://www.fhfa.gov/DataTools/Downloads/Documents/HPI/HPI_dictionary.xls'
r = requests.get(dictionary_url)
with open('data/hpi_dictionary.xls', 'wb') as f:
    f.write(r.content)


specifications_url = 'https://www.fhfa.gov/DataTools/Downloads/Documents/HPI/HPI_specifications.xls'
r = requests.get(specifications_url)
with open('data/hpi_specifications.xls', 'wb') as f:
    f.write(r.content)

In [25]:
df_specifications = pd.read_excel(specifications_url)

# Print text in row 1 column 0
print(df_specifications.iloc[1,0])

Notes/Abbreviations:
  PO = Purchase-Only Index; Index estimated exclusively with data from purchase-money mortgages. This is the traditional index.
  AT = All-Transactions Index estimated with appraisal values and sales prices from purchase-money mortgages
  ED = Expanded-Data Index; Index estimated with sales prices from purchase-money mortgages using Enterprise, FHA, and Real Property County Recorder Data
  DF = Distress-Free Index; Index estimated exclusively with data from purchase-money mortgages after removing distressed sales
  NSA = Not Seasonally Adjusted [If not specified, data series is NSA]
  SA = Seasonally Adjusted
  TXT = Tab-Delimited File
  XLS / XLSX = Excel File
  Census Divisions: DV_PAC=Pacific; DV_MT=Mountain; DV_WNC=West North Central; DV_WSC=West South Central; DV_ENC=East North Central; DV_ESC=East South Central; DV_NE=New England; DV_MA=Middle Atlantic; DV_SA=South Atlantic; USA=United States.


Explore the dataset features.

In [19]:
print('HPI Type: ', df_raw['hpi_type'].unique())

print('HPI Flavor: ', df_raw['hpi_flavor'].unique())

print('Frequency: ', df_raw['frequency'].unique())

print('Level: ', df_raw['level'].unique())

HPI Type:  ['traditional' 'non-metro' 'distress-free' 'developmental']
HPI Flavor:  ['purchase-only' 'all-transactions' 'expanded-data']
Frequency:  ['monthly' 'quarterly']
Level:  ['USA or Census Division' 'MSA' 'State' 'Puerto Rico']


We will use the following features:
- `traditional`
- `all-transactions` - Uses appraisal values and sales prices from purchase-money mortgages.
- `quarterly` - Released Tue 28th November, so we'll use this instead of monthly (can switch back to monthly once quarterly is out of date)
- `MSA` - Metropolitan Statistical Area

Multiple geographical levels, we'll go with the most granular: Metropolitan Statistical Area (`MSA`).

Quarterly data came out on Tue 28th Nov, so we'll use this instead of monthly

Refine the selection, starting from MSA.

In [178]:
df = df_raw[df_raw['level'] == 'MSA'].copy()
df = df[df['frequency'] == 'quarterly'].copy()
df = df[df['hpi_flavor'] == 'all-transactions'].copy()

# df['hpi_type'].unique()
df = df[df['hpi_type'] == 'traditional'].reset_index(drop=True).copy()

Check we have refined the dataset to the correct features.

In [179]:
print('Invalid feature combination') if len(df) == 0 else None
for col in ['hpi_type', 'hpi_flavor', 'frequency', 'level']:
    if len(df[col].unique()) > 1:
        print('Refine selection')
        print(col, df[col].unique())

In [180]:
# Add a date column using columns 'yr' and 'period', in format yyyy-mm-dd
# Use Lambda function, period is 1, 2, 3, 4, corresponding to Q1, Q2, Q3, Q4, so convert to 01-01, 04-01, 07-01, 10-01, ensure 2 digit month and day
df['date'] = df.apply(lambda x: str(x['yr']) + '-' + str(x['period'] * 3 - 2).zfill(2) + '-01', axis=1)

# # Convert date to datetime
# df['date'] = pd.to_datetime(df['date'])

We need to calculate the number of time periods (for either months or quarters) that every observation is away from the current time period. 

We could use a groupby & cumulative count, but if there are any missing observations at particular time periods, then our distance calculation will be wrong.

Instead, let's create some helper functions to calculate the distance between two dates, and then apply this to every observation in the dataset.

In [198]:
def month_diff(a, b) -> int:
    # Convert to datetime if not already
    a = datetime.strptime(a, '%Y-%m-%d') if not isinstance(a, datetime) else a
    b = datetime.strptime(b, '%Y-%m-%d') if not isinstance(b, datetime) else b
    return int((a.year - b.year) * 12 + a.month - b.month)

def quarter_diff(a, b) -> int:
    # Convert to datetime if not already
    a = datetime.strptime(a, '%Y-%m-%d') if not isinstance(a, datetime) else a
    b = datetime.strptime(b, '%Y-%m-%d') if not isinstance(b, datetime) else b
    return int((a.year - b.year) * 4 + (a.month - b.month) / 3)

print('Months: ', month_diff(current_period, previous_period))
print('Quarters: ', quarter_diff(current_period, previous_period))

Months:  12
Quarters:  4


In [200]:
df['distance'] = df['date'].apply(lambda x: quarter_diff(current_period, x))

Now, using the `distance` column, we can filter the dataset to only include observations `x` many time periods away from the current time period. 

E.g. if we have quarterly data and want to look at the 12-month change, filter the dataset so `distance` <= 4.

In [164]:
# If any columns are all null, drop them
df = df.dropna(axis=1, how='all')

---

In [122]:
df

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,date,distance
0,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1986,2,107.55,1986-04-01,149
1,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1986,3,107.84,1986-07-01,148
2,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1986,4,94.27,1986-10-01,147
3,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1987,1,100.73,1987-01-01,146
4,traditional,all-transactions,quarterly,MSA,"Abilene, TX",10180,1987,2,100.40,1987-04-01,145
...,...,...,...,...,...,...,...,...,...,...,...
65306,traditional,all-transactions,quarterly,MSA,"Yuma, AZ",49740,2022,3,312.90,2022-07-01,4
65307,traditional,all-transactions,quarterly,MSA,"Yuma, AZ",49740,2022,4,313.25,2022-10-01,3
65308,traditional,all-transactions,quarterly,MSA,"Yuma, AZ",49740,2023,1,298.53,2023-01-01,2
65309,traditional,all-transactions,quarterly,MSA,"Yuma, AZ",49740,2023,2,317.09,2023-04-01,1


Let's define some functions for analysing the data. 

We want to know which MSAs are having the biggest (and smallest) changes in house prices, over multiple time periods.

In [238]:
def get_trends(df: pd.DataFrame, periods: int, top: int = 5):
    '''Return MSAs with the largest increase and decrease in HPI over the specified number of periods'''

    # Filter for current period and period specified
    # 1. Get most recent period, then remove MSAs with no data for that period
    current_period = df['date'].max()
    # non_current = []
    # for msa in df['place_name'].unique():
    #     if df[df['place_name'] == msa]['date'].max() != current_period:
    #         non_current.append(msa)

    # df = df[~df['place_name'].isin(non_current)].copy()

    # 2. Filter `distance`
    df = df[df['distance'].isin([0, periods])].copy()
    start_period = df['date'].min()

    # Unstack the dataframes so we have a column for current period and a column for the period specified
    df = df.pivot(index='place_name', columns='distance', values='index_nsa').reset_index().rename_axis(None, axis=1)

    # For each MSA, calculate the percent change in HPI between the current and previous period
    df['change'] = (df[0] - df[periods]) / df[periods]

    # If an MSA has a missing observation for either period, `change` will be null, so drop those rows
    df = df.dropna(subset=['change']).copy()

    # Replace column names `0` and `periods` with actual dates
    df = df.rename(columns={0: current_period, periods: start_period})

    # Sort by the percent change in HPI
    df = df.sort_values('change', ascending=False).reset_index(drop=True)

    # Create a column for the rank of each MSA
    df['rank'] = df.index + 1

    # Create dfs with top and bottom MSAs
    df_top = df.head(top).reset_index(drop=True).copy()
    df_bottom = df.tail(top).reset_index(drop=True).copy()

    # Calculate the average growth rate for all MSAs
    # Define row to hold average
    row = {
        'place_name': 'Median',
        current_period: df[current_period].median(),
        start_period: df[start_period].median(),
        'change': df['change'].median(),
        'rank': df['rank'].median()
    }

    # Build dataframe with average
    dfs = [df_top, pd.DataFrame([row]), df_bottom]

    # Return the top and bottom MSAs, and the average growth rate
    return pd.concat(dfs).reset_index(drop=True)
    # return df_top, df_bottom, average

In [239]:
df_top = get_trends(df, 4, 5)

In [240]:
df_top

,place_name,2023-07-01,2022-07-01,change,rank
0,"Parkersburg-Vienna, WV",275.80,233.45,0.181409,1.0
1,"Sebring-Avon Park, FL",438.94,378.54,0.159560,2.0
2,"Mansfield, OH",272.16,235.29,0.156700,3.0
3,"Brunswick, GA",360.78,314.96,0.145479,4.0
4,"Dothan, AL",256.61,224.62,0.142418,5.0
5,Median,320.38,299.20,0.058170,202.0
6,"Idaho Falls, ID",403.36,420.64,-0.041080,399.0
7,"Twin Falls, ID",395.37,417.65,-0.053346,400.0
8,"Santa Rosa-Petaluma, CA",386.00,408.00,-0.053922,401.0
9,"Lake Charles, LA",256.47,276.88,-0.073714,402.0


---

### Chart: Top & Bottom MSAs by 12-month House Price Change

Let's clean up the names of the MSAs, so they are easier to read.

In [280]:
name_replace = {
    'Austin-Round Rock-Georgetown, TX': 'Greater Austin, TX'
}

In [302]:
importlib.reload(eco_styles)
styles = eco_styles.EcoStyles()
styles.register_and_enable_theme(dark_mode=True)

df_temp = get_trends(df, 4, 5)
value = 'change'

df_temp['place_name'] = df_temp['place_name'].replace(name_replace)

# Add a conditional colour column to indicate whether the LAD is low, average, or high
hex_low, hex_neutral, hex_high = '#36b7b4', '#596870', '#e6224b'
# hex high if value in top half of table, low if in bottom half, neutral if in neither
df_temp['colour'] = df_temp[value].apply(lambda x: hex_high if x > df_temp[value].median() else hex_low if x < df_temp[value].median() else hex_neutral)


title = alt.Title(
    'US House Price Growth',
    subtitle=['Top/bottom metropolitan areas for 12-month all-transaction house price growth', 'Q3 2022-23 | Source: FHFA HPI'],
    dx=5

)
chart = alt.Chart(df_temp, title=title).mark_bar().encode(
    y=alt.X('place_name:N', sort='-x', axis=alt.Axis(
        # bold label for median
        # labelFontWeight=alt.condition(alt.datum.place_name == 'Median', alt.value('bold'), alt.value('normal')),
        labelBound=False,
        labelLimit=220
    )),
    x=alt.Y(f'{value}:Q', axis=alt.Axis(format='%')),
    color=alt.Color('colour', scale=None)
)

# Add text to median bar
# median = df_temp[df_temp['place_name'] == 'Median'][value].values[0]
text = alt.Chart(df_temp[df_temp['place_name'] == 'Median']).mark_text(
    align='left',
    baseline='middle',
    dx=3,
    color='#b4c8d8',
    fontSize=14,
    text=[f'Median: {median:.1%}']
).encode(
    text=alt.Text(f'{value}:Q', format='.1%')
)

# Add a line at x=0
rule = alt.Chart(pd.DataFrame({'x': [0]})).mark_rule(color='#b4c8d8e6', size=2, xOffset=0.5).encode(x='x')

chart = chart + text + rule

chart.display()

alt.LayerChart(...)

In [303]:
from datetime import date
# Get today's date in formats YYYYMM & YYYYMMDD
yearmonth = date.today().strftime("%Y%m")
today = date.today().strftime("%Y%m%d")

styles.save(chart, path=f'../../charts/{yearmonth}', name=f'{today}_US_HPI_Growth', width=380)